# Jupyter meets MathBox2

from https://nbviewer.jupyter.org/github/znah/mathbox/blob/jupyter/examples/notebooks/mathbox.ipynb:
using [MathBox2](https://github.com/unconed/mathbox)

In [4]:
import json
import numpy as np
from IPython.display import HTML, Javascript, display

In [5]:
def json_numpy_serialzer(o):
    '''Helper function to serialize NumPy arrays.'''
    if isinstance(o, np.ndarray):
        return o.tolist()
    raise TypeError("{} of type {} is not JSON serializable".format(repr(o), type(o)))

def jsglobal(**params):
    '''Populate JS global namespace with provided Python obejcts.'''
    code = [];
    for name, value in params.items():
        jsdata = json.dumps(value, default=json_numpy_serialzer)
        code.append("window.{} = {};".format(name, jsdata))
    display(Javascript("\n".join(code)))

In [6]:
%%javascript

// Loading the compiled MathBox bundle.
require.config({
    paths: {
        mathBox: '//cdn.rawgit.com/unconed/mathbox/eaeb8e15/build/mathbox-bundle'
    }
});

// Helper function that setups WebGL context and initializes MathBox.
window.with_mathbox = function(element, func) {
    require(['mathBox'], function(){
        var mathbox = mathBox({
          plugins: ['core', 'controls', 'cursor', 'mathbox'],
          controls: { klass: THREE.OrbitControls },
          mathbox: {inspect: false},
          element: element[0],
          loop: {start: false},
            
        });
        var three = mathbox.three;
        three.renderer.setClearColor(new THREE.Color(0xFFFFFF), 1.0);
        three.camera.position.set(-1, 1, 2);
        three.controls.noKeys = true;
        
        three.element.style.height = "400px";
        three.element.style.width = "100%";
        
        function isInViewport(element) {
          var rect = element.getBoundingClientRect();
          var html = document.documentElement;
          var w = window.innerWidth || html.clientWidth;
          var h = window.innerHeight || html.clientHeight;
          return rect.top < h && rect.left < w && rect.bottom > 0 && rect.right > 0;
        }
        
        // Running update/render loop only for visible plots.
        var intervalId = setInterval(function(){
            if (three.element.offsetParent === null) {
                clearInterval(intervalId);
                three.destroy();
                return;
            }
            var visible = isInViewport(three.canvas);
            if (three.Loop.running != visible) {
                visible? three.Loop.start() : three.Loop.stop();
            }
        }, 100);

        func(mathbox);
        
        window.dispatchEvent(new Event('resize'));
    })
}

<IPython.core.display.Javascript object>

## Simple surface plot

This code snippet shows an 3d surface plot of a function, defined in JS callback.

In [293]:
%%javascript
function fn(i, j, k) {
    var rad = j / 20.0 * Math.PI / 3.0
    var y_centre = Math.sin(Math.PI/3) /3
    if(k==0) {
        var u = Math.cos(rad)-0.5
        var v = Math.sin(rad)-y_centre
    }
    else if(k==1) {
        var u = 0.5-Math.cos(rad)
        var v = Math.sin(rad)-y_centre
    }
    else if(k==2) {
        var u = Math.sin(rad - Math.PI/6)
        var v = y_centre*2-Math.cos(rad - Math.PI/6)
    }
    var az = i / 50.0 * Math.PI * 2.0 
    var r = Math.sqrt(u*u+v*v)
    var th = Math.atan2(v,u) + az*2/3 //twist it!
    var rr = 1.1
    var x = Math.sin(az+ Math.PI/3) * (1+r*Math.cos(th)*rr)
    var y = Math.cos(az+ Math.PI/3) * (1.1+r*Math.cos(th)*rr)
    var z = r*Math.sin(th)*rr
    return {x: x, y: z, z: y}
}

with_mathbox(element, function(mathbox) {
    var view = mathbox.cartesian({scale: [0.1, 0.1, 0.1]})//,{rotation:(t)=>[0, t*0.1, 0]}) // Setup rotating the coordinate frame.
    view.voxel({
        width: 50,
        height: 20,
        depth: 3,
        expr: function (emit, i, j, k, t) {
            var p0 = fn(i,j,k); emit(p0.x, p0.y, p0.z)
            var p1 = fn(i+1,j,k); emit(p1.x, p1.y, p1.z);
            var p2 = fn(i+1,j+1,k); emit(p2.x, p2.y, p2.z);
            var p3 = fn(i,j+1,k); emit(p3.x, p3.y, p3.z);
      },
      items: 4, channels: 3,
    })
    for(var i=-2; i<=2; ++i)
    for(var j=-2; j<=2; ++j)
        view.transform({position: [i*3.5, 0, j*3.5]})
            .transform({rotation: [(i+2)/Math.PI, 0, (j+2)/5.0]})
            .face({color: 0x3090FF, shaded: true})
}) 


<IPython.core.display.Javascript object>